## Hyperparameter tuning and ensemble of XGB, CatB, LGBM

This notebook contains 
* Hyperparamter tuning of XGBoost, CatBoost, LightBGM with KFold cross validation.
* Weighted ensemble of predictions obtained from tuned models with weights as reciprocal of individual scores. (This gave a score of **7.87595** on public LB data)

References:
* https://www.kaggle.com/c/tabular-playground-series-aug-2021/discussion/258009

## General Imports

In [ ]:
# General imports
import os
import cv2
import glob 
import json
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.svm import *
from sklearn.tree import *
from sklearn.metrics import *
from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.decomposition import *
from sklearn.preprocessing import *
from sklearn.model_selection import *

SEED = 42
np.random.seed(SEED)
sns.set_style("dark")
mpl.rcParams['figure.dpi'] = 200
%matplotlib inline

## Helper functions

In [ ]:
def rmse(preds, y):
    mse = mean_squared_error(preds, y)
    return np.sqrt(mse)

def write_subfile(reg, scaler=None, filename="submission.csv"):
    test_data = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")
    ids = test_data["id"]
    test_data = test_data.drop(columns=["id"])
    if scaler:
        test_data = scaler.transform(test_data)
    y = reg.predict(test_data)
    df = pd.DataFrame({
        "id" : ids,
        "loss" : y
    })
    df.to_csv(filename, index=False)

## Reading and scaling data

In [ ]:
data = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")
data.head()

In [ ]:
data.describe()

In [ ]:
Y = data["loss"]
X = data.drop(columns=["id", "loss"])

scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X) 

X_scaled.shape, Y.shape

In [ ]:
import optuna
from optuna.samplers import TPESampler

optuna.logging.set_verbosity(optuna.logging.WARNING)

## Tuning XGB Regressor using Optuna

In [ ]:
from xgboost import XGBRegressor

# Setup XGB hyperparameters for exps
def get_xgb_hyperparams(trail):
    xgb_params = {
        'learning_rate': 0.01,
        'tree_method': 'gpu_hist',
        'booster': 'gbtree',
        'n_estimators': trail.suggest_int('n_estimators', 500, 2500, 100),
        'reg_lambda': trail.suggest_int('reg_lambda', 1, 100),
        'reg_alpha': trail.suggest_int('reg_alpha', 1, 100),
        'subsample': trail.suggest_float('subsample', 0.2, 1.0, step=0.1),
        'colsample_bytree': trail.suggest_float('colsample_bytree', 0.2, 1.0, step=0.1),
        'max_depth': trail.suggest_int('max_depth', 3, 10), 
        'min_child_weight': trail.suggest_int('min_child_weight', 2, 10),
        'gamma': trail.suggest_float('gamma', 0, 20)        
    }
    return xgb_params

In [ ]:
# Define objective function
def objective_xgb(trail, X, Y, n_splits=4):
       
    xgb_params = get_xgb_hyperparams(trail)
    
    kfolds = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    total_loss = 0
    
    for train_index, val_index in kfolds.split(X):
        x_train, x_val = X[train_index], X[val_index]
        y_train, y_val = Y[train_index], Y[val_index]
    
        xgb_reg = XGBRegressor(**xgb_params)
    
        xgb_reg = xgb_reg.fit(x_train, y_train)
        preds = xgb_reg.predict(x_val)
    
        total_loss += rmse(preds, y_val)
    
    return total_loss / n_splits

In [ ]:
# Callback function to print log messages when the best trail is updated

def logging_callback(study, frozen_trail):
    prev_best = study.user_attrs.get('prev_best', None)
    if prev_best != study.best_value:
        study.set_user_attr('prev_best', study.best_value)
        print(f"Trail {frozen_trail.number} finished with best value {frozen_trail.value}")

In [ ]:
%%time

study = optuna.create_study(sampler=TPESampler(seed=SEED), 
                            direction='minimize', 
                            study_name='xgb_tuning')
objc = lambda trail : objective_xgb(trail, X_scaled, Y)

study.optimize(objc, n_trials=500, timeout=60*60, callbacks=[logging_callback])

In [ ]:
print(f"Best rmse value: {study.best_value}")
print(f"Best params: ")
for param, value in study.best_params.items():
    print(f"\t{param} : {value}")

In [ ]:
xgb_reg = XGBRegressor(**study.best_params,
                      learning_rate= 0.01,
                      tree_method= 'gpu_hist',
                      booster= 'gbtree')

xgb_reg.fit(X_scaled, Y, eval_metric='rmse', verbose=True)

print(f"RMSE on training data : {rmse(Y, xgb_reg.predict(X_scaled))}")

In [ ]:
write_subfile(xgb_reg, scaler, "xgb_tuned.csv") # score: 7.87652

## Tuning Cat Boost Regressor using Optuna

In [ ]:
from catboost import CatBoostRegressor

# Setup CatB hyperparameters for exps
def get_catb_hyperparams(trail):
    catb_params = {
        'loss_function': 'RMSE',
        'task_type': 'GPU',
        'eval_metric': 'RMSE',
        'bootstrap_type': 'Bernoulli',
        'iterations': trail.suggest_int('iterations', 1000, 5000),
        'od_wait': trail.suggest_int('od_wait', 500, 2000),
        'learning_rate': trail.suggest_uniform('learning_rate', 0.01, 0.5),
        'reg_lambda': trail.suggest_uniform('reg_lambda', 1e-4, 100),
        'subsample': trail.suggest_uniform('subsample', 0, 1),
        'random_strength': trail.suggest_uniform('random_strength', 10, 50),
        'depth': trail.suggest_int('depth', 1, 15),
        'min_data_in_leaf': trail.suggest_int('min_data_in_leaf', 1, 30),
        'leaf_estimation_iterations': trail.suggest_int('leaf_estimation_iterations', 1, 15)
    }
    return catb_params

In [ ]:
# Define objective function
def objective_catb(trail, X, Y, n_splits=4):
    
    catb_params = get_catb_hyperparams(trail)
    
    kfolds = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    total_loss = 0
    
    for train_index, val_index in kfolds.split(X):
        x_train, x_val = X[train_index], X[val_index]
        y_train, y_val = Y[train_index], Y[val_index]
    
        catb_reg = CatBoostRegressor(**catb_params)
    
        catb_reg = catb_reg.fit(x_train, y_train)
    
        preds = catb_reg.predict(x_val)
    
        total_loss += rmse(preds, y_val)
    
    return total_loss / n_splits

In [ ]:
%%time

study = optuna.create_study(sampler=TPESampler(seed=SEED), 
                            direction='minimize', 
                            study_name='catb_tuning')
objc = lambda trail : objective_catb(trail, X_scaled, Y)

study.optimize(objc, timeout=60*5, callbacks=[logging_callback])

In [ ]:
print(f"Best rmse value: {study.best_value}")
print(f"Best params: ")
for param, value in study.best_params.items():
    print(f"\t{param} : {value}")

In [ ]:
catb_reg = CatBoostRegressor(**study.best_params,
                            loss_function='RMSE',
                            task_type='GPU',
                            eval_metric='RMSE',
                            bootstrap_type='Bernoulli')

catb_reg.fit(X_scaled, Y)

print(f"RMSE on training data : {rmse(Y, catb_reg.predict(X_scaled))}")

In [ ]:
write_subfile(catb_reg, scaler, "catb_tuned.csv") # score: 7.89228

## Tuning Light GBM Regressor using Optuna

In [ ]:
from lightgbm import LGBMRegressor

# Setup lgbm hyperparameters for exps
def get_lgbm_hyperparams(trail):
    lgbm_params = {
        "objective": "rmse",
        "metric": "rmse",
        "boosting_type": "gbdt",
        "learning_rate": 0.008,
        'device': 'gpu',
        'n_estimators': trail.suggest_int("n_estimators", 500, 4000),
        "num_leaves": trail.suggest_int("num_leaves", 8, 256),
        "min_child_samples": trail.suggest_int("min_child_samples", 2, 3000),
        'feature_fraction': trail.suggest_uniform('feature_fraction', 0.25, 0.7),
        'bagging_fraction': trail.suggest_uniform('bagging_fraction', 0.7, 1.0),
        'bagging_freq': trail.suggest_int('bagging_freq', 0, 5),
        'reg_alpha': trail.suggest_int("reg_alpha", 1, 100),
        'reg_lambda': trail.suggest_int("reg_lambda", 1, 100),
    }
    return lgbm_params

In [ ]:
# Define objective function
def objective_lgbm(trail, X, Y, n_splits=4):

    lgbm_params = get_lgbm_hyperparams(trail)
    
    kfolds = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    total_loss = 0
    
    for train_index, val_index in kfolds.split(X):
        x_train, x_val = X[train_index], X[val_index]
        y_train, y_val = Y[train_index], Y[val_index]
    
        lgbm_reg = LGBMRegressor(**lgbm_params)
    
        lgbm_reg.fit(x_train, y_train)
    
        preds = lgbm_reg.predict(x_val)
    
        total_loss += rmse(preds, y_val)
    
    return total_loss / n_splits

In [ ]:
%%time

study = optuna.create_study(sampler=TPESampler(seed=SEED), 
                            direction='minimize', 
                            study_name='lgbm_tuning')
objc = lambda trail : objective_lgbm(trail, X_scaled, Y)

study.optimize(objc, timeout=60*60, callbacks=[logging_callback])

In [ ]:
print(f"Best value (rmse): {study.best_value:.5f}")
print(f"Best params:")
for key, value in study.best_params.items():
    print(f"\t{key}: {value}")

In [ ]:
lgbm_reg = LGBMRegressor(**study.best_params,
                        objective="rmse",
                        metric="rmse",
                        boosting_type="gbdt",
                        learning_rate=0.008,
                        device='gpu')

lgbm_reg.fit(X_scaled, Y, eval_metric='rmse', verbose=True)

print(f"RMSE on training data : {rmse(Y, lgbm_reg.predict(X_scaled))}")

In [ ]:
write_subfile(lgbm_reg, scaler, "lgbm_tuned.csv") # score: 7.89669

## Ensembling

In [ ]:
xgb_preds = pd.read_csv('xgb_tuned.csv')['loss']
catb_preds = pd.read_csv('catb_tuned.csv')['loss']
lgbm_preds = pd.read_csv('lgbm_tuned.csv')['loss']

scores = [7.87652, 7.89228, 7.87669]

weights = [1 / score for score in scores]
weights = np.array([weight / sum(weights) for weight in weights])

preds = np.array([xgb_preds, catb_preds, lgbm_preds])

ensemble_preds = np.zeros_like(xgb_preds)

for pred, weight in zip(preds, weights):
    ensemble_preds += weight * pred
    
ensemble_csv = pd.read_csv('xgb_tuned.csv') # Score: 7.87595
ensemble_csv['loss'] = ensemble_preds
ensemble_csv.to_csv('ensemble.csv', index=False)